This notebook is for the purpose of converting the essays in string form to vectorized form. This notebook makes use of the GLoVe word embeddings. More information about these word embeddings can be found here: https://nlp.stanford.edu/projects/glove/

If you'd like to move directly to the Neural Network, you can skip this notebook completely and load the prepared data directly!

If you would like to follow along with the preprocessing work, follow the link above and download the glove.6b.zip file to the "data/" directory of this project. Then uncompress the zip file to reveal directory named "glove.6B/" with 4 txt files inside. These are the four available word embeddings. You are not able to proceed with this notebook. 

In [13]:
from src.preprocess import get_data, vectorize_essays, normalize_scores
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Import training data from tsv file and store in a pandas dataframe
data_path = './data/training_set_rel3.tsv'
essay_df = get_data(data_path)

We can inspect the contents of the dataframe to understand what information is provided in the training set
More EDA if we have time

In [5]:
essay_df = essay_df[essay_df['essay_set']<=6]
essays = essay_df['essay']
essay_df['essays_embed'] = vectorize_essays(essays, 100, verbose=True)

The total number of essays is 10684


1000 Essays Vectorized


2000 Essays Vectorized


3000 Essays Vectorized


4000 Essays Vectorized
5000 Essays Vectorized


...

8000 Essays Vectorized


9000 Essays Vectorized


10000 Essays Vectorized


10684 Total Essays Vectorized!


/anaconda3/envs/EssayGrader_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
for set in range(1,7):
    if set==2:
        essay_df[essay_df['essay_set']]==set
    else:
        scores = essay_df[essay_df['essay_set']==set]['domain1_score']
        maximum = max(scores)
        norm_scores = normalize_scores(scores, maximum)

#essay_df['norm_scores'] = normalize_scores()

In [27]:
essay_df[domain]

Index(['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1',
       'rater3_domain1', 'domain1_score', 'rater1_domain2', 'rater2_domain2',
       'domain2_score', 'rater1_trait1', 'rater1_trait2', 'rater1_trait3',
       'rater1_trait4', 'rater1_trait5', 'rater1_trait6', 'rater2_trait1',
       'rater2_trait2', 'rater2_trait3', 'rater2_trait4', 'rater2_trait5',
       'rater2_trait6', 'rater3_trait1', 'rater3_trait2', 'rater3_trait3',
       'rater3_trait4', 'rater3_trait5', 'rater3_trait6', 'essays_embed'],
      dtype='object')

In [11]:
# Store the preprocessed data frame for later use
essay_df.to_pickle('./data/essay_df.pkl')